In [1]:
from ppi_plusplus_multi import *

import pandas as pd
import numpy as np
import os
# folder_dir = "../mexico_predictions/"
# site_name = "mexico"
# # mod_name = "gpt4zs"
# mod_name = "bert"
# file_name = folder_dir + site_name + "_"+ mod_name + ".csv"
# df = pd.read_csv(file_name)

In [2]:
df = pd.read_csv('../data/results/mexico_KNN.csv')

In [3]:
df.shape

(1306, 3)

## Mexico

In [4]:
# split the df into 20%(labeled) and 80% (unlabeled)
# stratified
# labeled_df = df.groupby('Y', group_keys=False).apply(lambda x: x.sample(frac=0.2))
# unlabeled_df = df.drop(labeled_df.index)

# split the df into 40%(labeled) and 60% (unlabeled)
# random
np.random.seed(123)
labeled_df = df.sample(frac=0.2)
unlabeled_df = df.drop(labeled_df.index)

In [47]:
Y = df['Y'].to_numpy()
X = df['X'].to_numpy()
X = X.reshape(-1,1)

sort_idx = np.argsort(Y)
Y_sorted = Y[sort_idx]
X_sorted = X[sort_idx]


import statsmodels.api as sm
mn_logit = sm.MNLogit(Y_sorted, X_sorted)
mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
mn_logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.961287
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                 1306
Model:                        MNLogit   Df Residuals:                     1302
Method:                           MLE   Df Model:                            0
Date:                Wed, 20 Mar 2024   Pseudo R-squ.:                 0.05837
Time:                        16:18:32   Log-Likelihood:                -1255.4
converged:                       True   LL-Null:                       -1333.3
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0125      0.003      4.846      0.000       0.007       0.018
------------------------------------------------------------------------------
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0040      0.003      1.404      0.160      -0.002       0.010
------------------------------------------------------------------------------
       y=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0294      0.005     -5.709      0.000      -0.039      -0.019
------------------------------------------------------------------------------
       y=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0429      0.002     19.309      0.000       0.039       0.047
==============================================================================
"""

In [48]:
Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat = labeled_df['Y_hat'].to_numpy()
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled = unlabeled_df['Y_hat'].to_numpy()

In [49]:
X_full = np.concatenate((X, X_unlabeled))
Y_full = np.concatenate((Y, Yhat_unlabeled))

In [50]:
Y = labeled_df['Y'].to_numpy()

sort_idx = np.argsort(Y)
Y_full_sorted = Y_full[sort_idx]
X_full_sorted = X_full[sort_idx]

In [51]:
mn_logit = sm.MNLogit(Y_full_sorted, X_full_sorted)
mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
mn_logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.901449
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                  261
Model:                        MNLogit   Df Residuals:                      257
Method:                           MLE   Df Model:                            0
Date:                Wed, 20 Mar 2024   Pseudo R-squ.:                 0.03177
Time:                        16:18:35   Log-Likelihood:                -235.28
converged:                       True   LL-Null:                       -243.00
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
       y=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0151      0.006      2.569      0.010       0.004       0.027
------------------------------------------------------------------------------
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0035      0.007     -0.476      0.634      -0.018       0.011
------------------------------------------------------------------------------
       y=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0204      0.010     -2.014      0.044      -0.040      -0.001
------------------------------------------------------------------------------
       y=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0448      0.005      8.646      0.000       0.035       0.055
==============================================================================
"""

In [25]:
theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:76: RuntimeWarning: overflow encountered in exp
  loss0 +=  -(Xi @ Ey) @ _theta + np.log(np.sum(np.exp(Xi @ EY @ _theta)))
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:80: RuntimeWarning: overflow encountered in exp
  loss1 += -(Xi @ Eyhat) @ _theta + np.log(np.sum(np.exp(Xi @ EY @ _theta)))
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:87: RuntimeWarning: overflow encountered in exp
  loss2 += -(Xi_unlabeled @ Ey_unlabeled) @ _theta + np.log(np.sum(np.exp(Xi_unlabeled @ EY @ _theta)))
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:90: RuntimeWarning: invalid value encountered in scalar subtract
  loss = 1 / n * loss0 - lhat_curr / n * loss1 + lhat_curr / N * loss2
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeW

/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:90: RuntimeWarning: invalid value encountered in scalar subtract
  loss = 1 / n * loss0 - lhat_curr / n * loss1 + lhat_curr / N * loss2
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_mu

/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:76: RuntimeWarning: overflow 

/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:87: RuntimeWarning: overflow encountered in exp
  loss2 += -(Xi_unlabeled @ Ey_unlabeled) @ _theta + np.log(np.sum(np.exp(Xi_unlabeled @ EY @ _theta)))
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:90: RuntimeWarning: invalid value encountered in scalar subtract
  loss = 1 / n * loss0 - lhat_curr / n * loss1 + lhat_curr / N * loss2
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=1, keepdims=True)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:107: RuntimeWarning: overflow encountered in exp
  a = np.exp(X @ _theta_2D.T)
/Users/adam/Desktop/Github/va_nlp/utils/ppi_plusplus_multi.py:108: RuntimeWarning: invalid value encountered in divide
  s = a / np.sum(a, axis=

ValueError: too many values to unpack (expected 4)

In [60]:
Y.shape

(261,)

In [51]:
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes:')
print(theta_ppi_ci['se'])

power tuning parameter value is:


NameError: name 'theta_ppi_ci' is not defined

## Pemba

In [8]:
folder_dir = "../classic_predictions/"
site_name = "pemba"
file_name = folder_dir + "classic_predictions_ex_" + site_name + ".csv"
df = pd.read_csv(file_name)

labeled_df = df.sample(frac=0.4)
unlabeled_df = df.drop(labeled_df.index)

Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat_NB = labeled_df['Yhat_NB'].to_numpy()
Yhat = Yhat_NB
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled_NB = unlabeled_df['Yhat_NB'].to_numpy()
Yhat_unlabeled = Yhat_unlabeled_NB

theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

In [9]:
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes when:')
print(theta_ppi_ci['se'])

power tuning parameter value is:
0.47697416050122554
parameter estimates: 
[-0.00264279 -0.01421964  0.01279802 -0.02854514]
prediction-powered confidence interval:
(array([-0.00371149, -0.01726476,  0.01186213, -0.03280631]), array([-0.00157409, -0.01117452,  0.01373391, -0.02428398]))
SE of PPI estiamtes when lambda = 0:
[0.00064972 0.0018513  0.00056898 0.00259061]


## ap

In [10]:
folder_dir = "../classic_predictions/"
site_name = "ap"
file_name = folder_dir + "classic_predictions_ex_" + site_name + ".csv"
df = pd.read_csv(file_name)

labeled_df = df.sample(frac=0.4)
unlabeled_df = df.drop(labeled_df.index)

Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat_NB = labeled_df['Yhat_NB'].to_numpy()
Yhat = Yhat_NB
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled_NB = unlabeled_df['Yhat_NB'].to_numpy()
Yhat_unlabeled = Yhat_unlabeled_NB

theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

In [11]:
print("The left-out site is:" + site_name)
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes when:')
print(theta_ppi_ci['se'])

The left-out site is:ap
power tuning parameter value is:
0.3078180666869222
parameter estimates: 
[-0.00624251 -0.00664452  0.02804172  0.00819674]
prediction-powered confidence interval:
(array([-0.00899999, -0.01039156,  0.02668012,  0.00667108]), array([-0.00348502, -0.00289748,  0.02940332,  0.00972241]))
SE of PPI estiamtes when:
[0.00167643 0.00227804 0.00082779 0.00092754]


## bohol

In [12]:
folder_dir = "../classic_predictions/"
site_name = "bohol"
file_name = folder_dir + "classic_predictions_ex_" + site_name + ".csv"
df = pd.read_csv(file_name)

labeled_df = df.sample(frac=0.4)
unlabeled_df = df.drop(labeled_df.index)

Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat_NB = labeled_df['Yhat_NB'].to_numpy()
Yhat = Yhat_NB
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled_NB = unlabeled_df['Yhat_NB'].to_numpy()
Yhat_unlabeled = Yhat_unlabeled_NB

theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

print("The left-out site is:" + site_name)
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes when:')
print(theta_ppi_ci['se'])

The left-out site is:bohol
power tuning parameter value is:
0.39727643474894964
parameter estimates: 
[-0.00361776 -0.03339858  0.02243895 -0.04621501]
prediction-powered confidence interval:
(array([-0.00494442, -0.04367511,  0.02125039, -0.05895053]), array([-0.00229111, -0.02312204,  0.02362751, -0.03347949]))
SE of PPI estiamtes when:
[0.00080655 0.00624769 0.00072259 0.00774265]


## dar

In [13]:
folder_dir = "../classic_predictions/"
site_name = "dar"
file_name = folder_dir + "classic_predictions_ex_" + site_name + ".csv"
df = pd.read_csv(file_name)

labeled_df = df.sample(frac=0.4)
unlabeled_df = df.drop(labeled_df.index)

Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat_NB = labeled_df['Yhat_NB'].to_numpy()
Yhat = Yhat_NB
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled_NB = unlabeled_df['Yhat_NB'].to_numpy()
Yhat_unlabeled = Yhat_unlabeled_NB

theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

print("The left-out site is:" + site_name)
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes when:')
print(theta_ppi_ci['se'])

The left-out site is:dar
power tuning parameter value is:
0.3613783126538752
parameter estimates: 
[-0.00527376 -0.02782146  0.03221836 -0.01039343]
prediction-powered confidence interval:
(array([-0.00749293, -0.03821043,  0.03071345, -0.01323097]), array([-0.0030546 , -0.01743249,  0.03372328, -0.0075559 ]))
SE of PPI estiamtes when:
[0.00134916 0.00631605 0.00091492 0.0017251 ]


## up

In [14]:
folder_dir = "../classic_predictions/"
site_name = "up"
file_name = folder_dir + "classic_predictions_ex_" + site_name + ".csv"
df = pd.read_csv(file_name)

labeled_df = df.sample(frac=0.4)
unlabeled_df = df.drop(labeled_df.index)

Y = labeled_df['Y'].to_numpy()
X = labeled_df['X'].to_numpy()
X = X.reshape(-1,1)
Yhat_NB = labeled_df['Yhat_NB'].to_numpy()
Yhat = Yhat_NB
X_unlabeled = unlabeled_df['X'].to_numpy()
X_unlabeled = X_unlabeled.reshape(-1,1)
Yhat_unlabeled_NB = unlabeled_df['Yhat_NB'].to_numpy()
Yhat_unlabeled = Yhat_unlabeled_NB

theta_ppi_ci = ppi_multiclass_logistic_ci(
            X,
            Y,
            Yhat,
            X_unlabeled,
            Yhat_unlabeled,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

print("The left-out site is:" + site_name)
print("power tuning parameter value is:")
print(theta_ppi_ci['lhat'])
print("parameter estimates: ")
print(theta_ppi_ci['pointest'])
print("prediction-powered confidence interval:")
print(theta_ppi_ci['ci'])
print('SE of PPI estiamtes when:')
print(theta_ppi_ci['se'])

The left-out site is:up
power tuning parameter value is:
0.4485352499454914
parameter estimates: 
[ 0.00577535 -0.00656915  0.00574502 -0.05527534]
prediction-powered confidence interval:
(array([ 0.00349284, -0.01143386,  0.00308815, -0.0980854 ]), array([ 0.00805785, -0.00170444,  0.00840188, -0.01246529]))
SE of PPI estiamtes when:
[0.00138767 0.00295753 0.00161526 0.02602667]
